In [10]:
import os
import re
from datetime import date, timedelta

import httpx
import pandas as pd

In [3]:
def create_client() -> httpx.Client:
    """
    Creates a common client for future http requests

    Returns:
        httpx.Client: client with ford proxies
    """
    ford_proxy = str(os.getenv("FORD_PROXY"))
    timeout_config = httpx.Timeout(10.0, connect=5.0)
    proxy_mounts = {
        "http://": httpx.HTTPTransport(proxy=httpx.Proxy(ford_proxy)),
        "https://": httpx.HTTPTransport(proxy=httpx.Proxy(ford_proxy)),
    }
    return httpx.Client(
        timeout=timeout_config,
        mounts=proxy_mounts,
        verify=False,
    )

In [14]:
tomorow = date.today() + timedelta(days=1)
dataset = []
with create_client() as client:
    response = client.get(
        str(os.getenv("NHTSA_RECALS_URL")),
        params={
            "query": (
                f"select * where `report_received_date` >= '{os.getenv('LAST_RECALL_WAVE_DATE')}'"
                + f" AND `report_received_date` < '{date.today().strftime('%Y-%m-%d')}'"
            ),
            "read_from_nbe": True,
            "version": 2.1,
            "date": tomorow.strftime("%Y-%m-%d"),
            "accessType": "DOWNLOAD",
        },
    )

rows = response.text.split("\n")
pattern = r',(?=(?:[^"]*"[^"]*")*[^"]*$)'

for i in range(1, len(rows)):
    values = re.split(pattern, rows[i])
    if len(values) <= 1:
        break  # the last row always come empty
    dataset.append(values)

data = pd.DataFrame(
    dataset,
    columns=[
        "REPORT_RECEIVED_DATE",
        "NHTSA_ID",
        "RECALL_LINK",
        "MANUFACTURER",
        "SUBJECT",
        "COMPONENT",
        "MFR_CAMPAIGN_NUMBER",
        "RECALL_TYPE",
        "POTENTIALLY_AFFECTED",
        "RECALL_DESCRIPTION",
        "CONSEQUENCE_SUMMARY",
        "CORRECTIVE_ACTION",
        "PARK_OUTSIDE_ADVISORY",
        "DO_NOT_DRIVE_ADVISORY",
        "COMPLETION_RATE",
    ],
)
data.head()

,REPORT_RECEIVED_DATE,NHTSA_ID,RECALL_LINK,MANUFACTURER,SUBJECT,COMPONENT,MFR_CAMPAIGN_NUMBER,RECALL_TYPE,POTENTIALLY_AFFECTED,RECALL_DESCRIPTION,CONSEQUENCE_SUMMARY,CORRECTIVE_ACTION,PARK_OUTSIDE_ADVISORY,DO_NOT_DRIVE_ADVISORY,COMPLETION_RATE
0,02/06/2024,24V072000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,"""Mitsubishi Fuso Truck of America, Inc.""",Engine May Shut Down Unexpectedly,ELECTRICAL SYSTEM,C10126,Vehicle,175,"""Mitsubishi Fuso Truck of America, Inc. (MFTA)...",A loss of drive power can increase the risk of...,"""Dealers will reprogram vehicle control unit a...",No,No,
1,02/06/2024,24E008000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,JTEKT,Improperly Tightened Bolts on Power Steering Unit,STEERING,NR (Not Reported),Equipment,423,"""JTEKT is recalling certain Electronic Power S...",A loss of steering control increases the risk ...,"""Dealers will replace the steering gear, free ...",No,No,
2,02/07/2024,24V081000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,"""Suzuki Motor USA, LLC""",Drive Chain May Detach,POWER TRAIN,3A02,Vehicle,360,"""Suzuki Motor USA, LLC (Suzuki) is recalling c...",A loss of drive power increases the risk of a ...,"""Dealers will inspect and repair the drive cha...",No,No,
3,02/07/2024,24V077000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,"""Kia America, Inc.""",Engine Valve Springs May Break,ENGINE AND ENGINE COOLING,SC296,Vehicle,2872,"""Kia America, Inc. (Kia) is recalling certain ...",A loss of drive power increases the risk of a ...,"""Dealers will replace the engine sub-assembly,...",No,No,
4,02/07/2024,24V075000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,"""Hino Motors Sales U.S.A., Inc.""",Improperly Secured Ground Connection,ELECTRICAL SYSTEM,M0460,Vehicle,12118,"""Hino Motors Sales U.S.A., Inc. (Hino) is reca...",An overheated ground connection can increase t...,"""Dealers will inspect and repair the ground ci...",No,No,
